# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [0]:
"""from google.colab import drive
drive.mount('/content/drive')"""

In [1]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "pcm" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "jw300-baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag


# This will save it to a folder in our gdrive instead!
!mkdir -p "en_pcm/$src-$tgt-$tag"
os.environ["experiment_path"] = "en_pcm/%s-%s-%s" % (source_language, target_language, tag)

In [2]:
!mkdir -p "en_pcm/$src-$tgt-$tag"

In [3]:
!echo $experiment_path

en_pcm/en-pcm-jw300-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q 

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz --y


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-pcm.xml.gz not found. The following files are available for downloading:

 256 KB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-pcm.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
   3 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/pcm.zip

 266 MB Total size
./JW300_latest_xml_en-pcm.xml.gz ... 100% of 256 KB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_pcm.zip ... 100% of 3 MB
gzip: unrecognized option '--y'
Try `gzip --help' for more information.


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-02-09 12:37:41--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.24.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.24.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.008s  

2020-02-09 12:37:41 (35.2 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-02-09 12:37:41--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-pcm.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.24.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.24.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154369 (151K) [text/plain]
Saving to: ‘test.en-pcm.en’

test.en-pcm

In [7]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [8]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 2536/26020 lines since contained in test set.


,source_sentence,target_sentence
0,3 Settle Differences in a Spirit of Love,3 Make Una Dey Use Love Settle Quarrel
1,"Because of our inherited imperfection , we are...",Because of the sin wey all of us carry from be...
2,This article shows how Bible principles can be...,This topic go show us how we fit let the thing...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [9]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 51kB 14.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for python-Levenshtein ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
00:00:00.05 0.00 percent complete
00:00:26.30 4.70 percent complete
00:00:52.73 9.40 percent complete
00:01:19.04 14.10 percent complete
00:01:45.75 18.79 percent complete
00:02:11.71 23.49 percent complete
00:02:38.52 28.19 percent complete
00:03:04.38 32.89 percent complete
00:03:31.22 37.59 percent complete
00:03:56.93 42.29 percent complete
00:04:23.57 46.99 percent complete


00:04:50.25 51.68 percent complete
00:05:15.71 56.38 percent complete
00:05:41.33 61.08 percent complete
00:06:07.34 65.78 percent complete
00:06:33.58 70.48 percent complete
00:06:59.31 75.18 percent complete
00:07:26.38 79.88 percent complete
00:07:53.15 84.57 percent complete
00:08:19.62 89.27 percent complete
00:08:45.67 93.97 percent complete


00:09:11.83 98.67 percent complete


In [11]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
JEHOVAH’S servants highly esteem God’s own holy book , the Bible .
At noon , they slowly walk home .
To put the woman at ease , Jesus kindly said : “ Take courage , daughter ! ”
( Compare Ezekiel 28 : 17 . )
God’s active force is a very powerful source of comfort .
What can we do to make wise use of our freedom ?
Are you young or up in years ?
“ People in our area are becoming more and more radical , ” notes one traveling overseer .
This set Adam apart from the animals , since they live according to instinct .
This , of course , was no accident .

==> train.pcm <==
JEHOVAH people value Bible well well because dem know sey na God holy book .
Around twelve o’clock , they go waka sofri - sofri go house .
Jesus come tell am wetin go make am no fear again . E tell am sey : ‘ No fear , my pikin ! ’
( Still check Ezekiel 28 : 17 . )
This one na one way wey e take dey help us .
Wetin go help us use our freedom well ?
You dey young or you don old ?
One circuit overseer talk say



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [12]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 2277 (delta 56), reused 47 (delta 26), pack-reused 2184
Receiving objects: 100% (2277/2277), 2.63 MiB | 5.06 MiB/s, done.
Resolving deltas: 100% (1577/1577), done.
Processing /home/ec2-user/SageMaker/masakhane/joeynmt
    100% |████████████████████████████████| 753.4MB 66kB/s  eta 0:00:01    26% |████████▌                       | 200.0MB 56.3MB/s eta 0:00:10    35% |███████████▎                    | 264.9MB 58.5MB/s eta 0:00:09    50% |████████████████▏               | 380.8MB 57.0MB/s eta 0:00:07    66% |█████████████████████▏          | 498.3MB 55.6MB/s eta 0:00:05    72% |███████████████████████▎        | 547.4MB 59.3MB/s eta 0:00:04    90% |█████████████████████████████   | 681.6MB 58.1MB/s eta 0:00:02
    100% |████████████████████████████████| 421.8MB 117kB/s eta 0:00:010:09    7% |██▍                        

    100% |████████████████████████████████| 2.9MB 16.4MB/s ta 0:00:01
    100% |████████████████████████████████| 307kB 39.7MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 36.3MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 37.0MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 40.1MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 40.4MB/s ta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-tl77crle/wheels/3a/30/44/d8a7e4bf0ed7c54548bf491738b185ef57222ad3bcdfb1afc4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [13]:
!sudo pip3 install subword-nmt

  The script subword-nmt is installed in '/usr/local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You are using pip version 19.0.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$experiment_path"
! cp test.* "$experiment_path"
! cp dev.* "$experiment_path"
! cp bpe.codes.4000 "$gexperiment_path"
! ls "$experiment_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Pidgin Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.pcm    test.pcm	    train.en
dev.bpe.en	dev.pcm      test.en	     train.bpe.en   train.pcm
dev.bpe.pcm	test.bpe.en  test.en-any.en  train.bpe.pcm
cp: cannot create regular file ‘’: No such file or directory
dev.bpe.en   dev.pcm	   test.en	   train.bpe.en   train.pcm
dev.bpe.pcm  test.bpe.en   test.en-any.en  train.bpe.pcm
dev.en	     test.bpe.pcm  test.pcm	   train.en
BPE Pidgin Sentences
( b ) Wetin you go get for mind sey you no go do ?
book . “ Na wetin I learn for Proverbs 27 : ​ 11 , Matthew 26 : ​ 52 , and John 13 : ​ 35 , give me mind wey mek I no join army .
And even when the wahala come , na wetin dey for this Bible verse dem , still help me fit bear the wahala . ” ​ — An@@ dri@@ y , wey come from U@@ k@@ ra@@ ine .
“ Na wetin dey Isaiah 2 : 4 help me mek I no follow fight war , even when dem want mek I join .
I just dey think for my mind how trouble no go dey the new world .
Combined BPE Vocab
ʺ
satisfac@@
righte@@
ʼ@@
selves
fts
circu@@
ga

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["experiment_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{experiment_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 200                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 5

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, experiment_path=os.environ["experiment_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

##### Extra Installs
Because I am running on AWS, I need to do some extra installs because of environment conflicts. 

In [16]:
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - cudatoolkit=10.1
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB
    pytorch-1.4.0              |py3.6_cuda10.1.243_cudnn7.6.3_0       432.9 MB  pytorch
    openssl-1.0.2u             |       h7b6447c_0         3.1 MB
    certifi-2019.11.28         |           py36_0         156 KB
    torchvision-0.5.0          |       py36_cu101         9.1 MB  pytorch
    ------------------------------------------------------------
                                           Total:       445

In [17]:
!conda install tensorboard --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - tensorboard


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    c-ares-1.15.0              |    h7b6447c_1001         102 KB
    absl-py-0.8.1              |           py36_0         161 KB
    markdown-3.1.1             |           py36_0         113 KB
    numpy-1.18.1               |   py36h4f9e942_0           5 KB
    scikit-learn-0.22.1        |   py36hd81dba3_0         7.1 MB
    tensorboard-2.0.0          |     pyhb38c66f_1         3.3 MB
    mkl-2020.0                 |              166       202.1 MB
    scipy-1.4.1                |   py36h0b6359f_0        18.9 M

In [18]:
!conda install -c pytorch torchtext --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - torchtext


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tqdm-4.42.0                |             py_0          56 KB
    torchtext-0.5.0            |             py_1         1.4 MB  pytorch
    ------------------------------------------------------------
                                           Total:         1.5 MB

The following NEW packages will be INSTALLED:

    torchtext: 0.5.0-py_1  pytorch
    tqdm:      4.42.0-py_0        


tqdm-4.42.0          | 56 KB     | ##################################### | 100% 
torchtext-0.5.0      | 1.4 MB    | ###############

In [19]:
!conda install -c powerai sentencepiece --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - sentencepiece


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astor-0.8.0                |           py36_0          45 KB
    tensorflow-2.0.0           |mkl_py36hef7ec59_0           3 KB
    sentencepiece-0.1.84       |   py36h6bb024c_0         3.1 MB  powerai
    keras-applications-1.0.8   |             py_0          33 KB
    tensorflow-base-2.0.0      |mkl_py36h9204916_0       100.9 MB
    gast-0.2.2                 |           py36_0         138 KB
    google-pasta-0.1.8         |             py_0          43 KB
    opt_einsum-3.1.0           |             py_0 

In [20]:
!conda install -c powerai sacrebleu --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - sacrebleu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    sacrebleu-1.4.3            |             py_0          34 KB  powerai
    portalocker-1.5.2          |           py36_0          20 KB
    ------------------------------------------------------------
                                           Total:          54 KB

The following NEW packages will be INSTALLED:

    portalocker: 1.5.2-py36_0        
    sacrebleu:   1.4.3-py_0   powerai


sacrebleu-1.4.3      | 34 KB     | ##################################### | 100% 
portalocker-1.5.2    | 20 KB     | #########

##### Train the model

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-02-09 13:08:37,412 Hello! This is Joey-NMT.
2020-02-09 13:08:40,281 Total params: 12099840
2020-02-09 13:08:40,283 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

2020-02-09 13:08:44,832 cfg.name                           : enpcm_transformer
2020-02-09 13:08:44,832 cfg.data.src                       : en
2020-02-09 13:08:44,832 cfg.data.trg                       : pcm
2020-02-09 13:08:44,832 cfg.data.train                     : data/enpcm/train.bpe
2020-02-09 13:08:44,832 cfg.data.dev                       : data/enpcm/dev.bpe
2020-02-09 13:08:44,832 cfg.data.test                      : data/enpcm/test.bpe
2020-02-09 13:08:44,832 cfg.data.level                     : bpe
2020-02-09 13:08:44,832 cfg.data.lowercase                 : False
2020-02-09 13:08:44,832 cfg.data.max_sent_length           : 100
2020-02-09 13:08:44,832 cfg.data.src_vocab                 : data/enpcm/vocab.txt
2020-02-09 13:08:44,833 cfg.data.trg_vocab                 : data/enpcm/vocab.txt
2020-02-09 13:08:44,833 cfg.testing.beam_size              : 5
2020-02-09 13:08:44,833 cfg.testing.alpha                  : 1.0
2020-02-09 13:08:44,833 cfg.training.random_seed           :

2020-02-09 13:10:51,461 Epoch   5: total training loss 712.52
2020-02-09 13:10:51,461 EPOCH 6
2020-02-09 13:10:53,275 Epoch   6 Step:     1100 Batch Loss:     3.265706 Tokens per Sec:    21850, Lr: 0.000300
2020-02-09 13:11:03,098 Epoch   6 Step:     1200 Batch Loss:     3.291728 Tokens per Sec:    21540, Lr: 0.000300
2020-02-09 13:11:12,990 Epoch   6 Step:     1300 Batch Loss:     2.943817 Tokens per Sec:    20834, Lr: 0.000300
2020-02-09 13:11:12,991 Epoch   6: total training loss 684.58
2020-02-09 13:11:12,991 EPOCH 7
2020-02-09 13:11:22,883 Epoch   7 Step:     1400 Batch Loss:     2.914759 Tokens per Sec:    21785, Lr: 0.000300
2020-02-09 13:11:32,637 Epoch   7 Step:     1500 Batch Loss:     3.037468 Tokens per Sec:    21424, Lr: 0.000300
2020-02-09 13:11:34,103 Epoch   7: total training loss 649.63
2020-02-09 13:11:34,104 EPOCH 8
2020-02-09 13:11:42,433 Epoch   8 Step:     1600 Batch Loss:     2.900786 Tokens per Sec:    21571, Lr: 0.000300
2020-02-09 13:11:52,268 Epoch   8 Step: 

2020-02-09 13:15:15,027 EPOCH 17
2020-02-09 13:15:19,493 Epoch  17 Step:     3500 Batch Loss:     2.162553 Tokens per Sec:    21523, Lr: 0.000300
2020-02-09 13:15:29,371 Epoch  17 Step:     3600 Batch Loss:     2.326906 Tokens per Sec:    21915, Lr: 0.000300
2020-02-09 13:15:36,243 Epoch  17: total training loss 491.85
2020-02-09 13:15:36,244 EPOCH 18
2020-02-09 13:15:39,246 Epoch  18 Step:     3700 Batch Loss:     2.380447 Tokens per Sec:    21543, Lr: 0.000300
2020-02-09 13:15:49,155 Epoch  18 Step:     3800 Batch Loss:     2.020103 Tokens per Sec:    21464, Lr: 0.000300
2020-02-09 13:15:57,601 Epoch  18: total training loss 487.15
2020-02-09 13:15:57,602 EPOCH 19
2020-02-09 13:15:59,043 Epoch  19 Step:     3900 Batch Loss:     2.084379 Tokens per Sec:    19882, Lr: 0.000300
2020-02-09 13:16:08,997 Epoch  19 Step:     4000 Batch Loss:     2.317714 Tokens per Sec:    21471, Lr: 0.000300
2020-02-09 13:16:19,854 Hooray! New best validation result [ppl]!
2020-02-09 13:16:19,855 Saving ne

2020-02-09 13:19:51,539 Epoch  28 Step:     6000 Batch Loss:     2.292132 Tokens per Sec:    22003, Lr: 0.000300
2020-02-09 13:20:03,538 Hooray! New best validation result [ppl]!
2020-02-09 13:20:03,539 Saving new checkpoint.
2020-02-09 13:20:03,726 Example #0
2020-02-09 13:20:03,726 	Source:     Jehovah did not do that , but he allowed Satan to test Job , stating : “ Everything that he has is in your hand . ”
2020-02-09 13:20:03,726 	Reference:  E tell am sey : ‘ Everything wey e get dey your hand . ’
2020-02-09 13:20:03,726 	Hypothesis: Jehovah no let Job do wetin e do . But e no do wetin e do . E sey : ‘ Na you be the God wey dey do you . ’
2020-02-09 13:20:03,726 Example #1
2020-02-09 13:20:03,726 	Source:     Corinna said : “ We left our work area in the evening and walked to a railway station 25 kilometers ( 15 miles ) away .
2020-02-09 13:20:03,727 	Reference:  Corinna talk sey : “ We comot for where we dey work for evening come trek go where people dey enter train wey be 25 kil

...

2020-02-09 14:11:57,207 Epoch 157: total training loss 166.36
2020-02-09 14:11:57,207 EPOCH 158
2020-02-09 14:12:03,045 Epoch 158 Step:    34000 Batch Loss:     0.839570 Tokens per Sec:    21424, Lr: 0.000103
2020-02-09 14:12:15,485 Example #0
2020-02-09 14:12:15,485 	Source:     Jehovah did not do that , but he allowed Satan to test Job , stating : “ Everything that he has is in your hand . ”
2020-02-09 14:12:15,485 	Reference:  E tell am sey : ‘ Everything wey e get dey your hand . ’
2020-02-09 14:12:15,485 	Hypothesis: Jehovah no let Satan do wetin e no like . E tell Job sey : ‘ Anybody wey dey do wetin e want , na im be your hand . ’
2020-02-09 14:12:15,486 Example #1
2020-02-09 14:12:15,486 	Source:     Corinna said : “ We left our work area in the evening and walked to a railway station 25 kilometers ( 15 miles ) away .
2020-02-09 14:12:15,486 	Reference:  Corinna talk sey : “ We comot for where we dey work for evening come trek go where people dey enter train wey be 25 kilometer

2020-02-09 14:16:08,591 Epoch 167 Step:    36100 Batch Loss:     0.651248 Tokens per Sec:    21533, Lr: 0.000072
2020-02-09 14:16:08,987 Epoch 167: total training loss 161.08
2020-02-09 14:16:08,988 EPOCH 168
2020-02-09 14:16:18,537 Epoch 168 Step:    36200 Batch Loss:     0.704232 Tokens per Sec:    21435, Lr: 0.000072
2020-02-09 14:16:28,401 Epoch 168 Step:    36300 Batch Loss:     0.684729 Tokens per Sec:    21561, Lr: 0.000072
2020-02-09 14:16:30,265 Epoch 168: total training loss 159.60
2020-02-09 14:16:30,265 EPOCH 169
2020-02-09 14:16:38,278 Epoch 169 Step:    36400 Batch Loss:     0.832892 Tokens per Sec:    21379, Lr: 0.000072
2020-02-09 14:16:48,172 Epoch 169 Step:    36500 Batch Loss:     0.603741 Tokens per Sec:    21275, Lr: 0.000072
2020-02-09 14:16:51,618 Epoch 169: total training loss 160.18
2020-02-09 14:16:51,618 EPOCH 170
2020-02-09 14:16:58,056 Epoch 170 Step:    36600 Batch Loss:     0.663849 Tokens per Sec:    21777, Lr: 0.000072
2020-02-09 14:17:07,895 Epoch 170 

2020-02-09 14:20:42,318 Epoch 179 Step:    38600 Batch Loss:     0.880630 Tokens per Sec:    21455, Lr: 0.000072
2020-02-09 14:20:51,587 Epoch 179: total training loss 158.21
2020-02-09 14:20:51,587 EPOCH 180
2020-02-09 14:20:52,223 Epoch 180 Step:    38700 Batch Loss:     0.715024 Tokens per Sec:    19638, Lr: 0.000072
2020-02-09 14:21:02,175 Epoch 180 Step:    38800 Batch Loss:     0.700444 Tokens per Sec:    21450, Lr: 0.000072
2020-02-09 14:21:12,087 Epoch 180 Step:    38900 Batch Loss:     0.672916 Tokens per Sec:    21168, Lr: 0.000072
2020-02-09 14:21:13,071 Epoch 180: total training loss 155.96
2020-02-09 14:21:13,072 EPOCH 181
2020-02-09 14:21:22,093 Epoch 181 Step:    39000 Batch Loss:     0.338834 Tokens per Sec:    21004, Lr: 0.000072
2020-02-09 14:21:34,349 Example #0
2020-02-09 14:21:34,349 	Source:     Jehovah did not do that , but he allowed Satan to test Job , stating : “ Everything that he has is in your hand . ”
2020-02-09 14:21:34,349 	Reference:  E tell am sey : ‘ 

2020-02-09 14:25:17,237 Example #0
2020-02-09 14:25:17,238 	Source:     Jehovah did not do that , but he allowed Satan to test Job , stating : “ Everything that he has is in your hand . ”
2020-02-09 14:25:17,238 	Reference:  E tell am sey : ‘ Everything wey e get dey your hand . ’
2020-02-09 14:25:17,238 	Hypothesis: Jehovah no let Satan do wetin e no like . E tell Job sey : ‘ Anybody wey dey do wetin e want , na im be the true God . ’
2020-02-09 14:25:17,238 Example #1
2020-02-09 14:25:17,238 	Source:     Corinna said : “ We left our work area in the evening and walked to a railway station 25 kilometers ( 15 miles ) away .
2020-02-09 14:25:17,238 	Reference:  Corinna talk sey : “ We comot for where we dey work for evening come trek go where people dey enter train wey be 25 kilometer ( 15 miles ) from where the farm dey .
2020-02-09 14:25:17,238 	Hypothesis: Corinna , e come sey : ‘ We comot for where we dey stay . We dey stay there for where we dey go . We dey go preach for there .
20

2020-02-09 14:29:02,928 Epoch 199: total training loss 150.54
2020-02-09 14:29:02,928 EPOCH 200
2020-02-09 14:29:09,841 Epoch 200 Step:    43100 Batch Loss:     0.733609 Tokens per Sec:    21135, Lr: 0.000050
2020-02-09 14:29:19,853 Epoch 200 Step:    43200 Batch Loss:     0.636086 Tokens per Sec:    21286, Lr: 0.000050
2020-02-09 14:29:24,549 Epoch 200: total training loss 149.87
2020-02-09 14:29:24,550 Training ended after 200 epochs.
2020-02-09 14:29:24,550 Best validation result (greedy) at step    10000:   8.84 ppl.
2020-02-09 14:29:39,550  dev bleu:  12.78 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-02-09 14:29:39,551 Translations saved to: models/enpcm_transformer/00010000.hyps.dev
2020-02-09 14:30:02,666 test bleu:  24.29 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-02-09 14:30:02,667 Translations saved to: models/enpcm_transformer/00010000.hyps.test


In [23]:
!mkdir -p "$experiment_path/models/${src}${tgt}_transformer/"

In [24]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$experiment_path/models/${src}${tgt}_transformer/"

In [25]:
# Output our validation accuracy
! cat "$experiment_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 73394.23438	PPL: 24.65521	bleu: 2.06072	LR: 0.00030000	*
Steps: 2000	Loss: 62560.94141	PPL: 15.36234	bleu: 4.45320	LR: 0.00030000	*
Steps: 3000	Loss: 56698.69531	PPL: 11.89271	bleu: 7.73914	LR: 0.00030000	*
Steps: 4000	Loss: 53580.67188	PPL: 10.37882	bleu: 8.94406	LR: 0.00030000	*
Steps: 5000	Loss: 51855.69531	PPL: 9.62574	bleu: 10.11587	LR: 0.00030000	*
Steps: 6000	Loss: 50861.65234	PPL: 9.21685	bleu: 11.02907	LR: 0.00030000	*
Steps: 7000	Loss: 50434.14062	PPL: 9.04638	bleu: 11.47610	LR: 0.00030000	*
Steps: 8000	Loss: 50059.23047	PPL: 8.89948	bleu: 12.05842	LR: 0.00030000	*
Steps: 9000	Loss: 50120.64844	PPL: 8.92338	bleu: 12.70072	LR: 0.00030000	
Steps: 10000	Loss: 49895.84375	PPL: 8.83621	bleu: 12.57659	LR: 0.00030000	*
Steps: 11000	Loss: 50398.80078	PPL: 9.03242	bleu: 12.10177	LR: 0.00030000	
Steps: 12000	Loss: 50478.47656	PPL: 9.06391	bleu: 12.89334	LR: 0.00030000	
Steps: 13000	Loss: 51401.37500	PPL: 9.43665	bleu: 12.72072	LR: 0.00030000	
Steps: 14000	Loss: 51182.

In [27]:
# Test our model
! cd joeynmt; python3 -m joeynmt test configs/transformer_$src$tgt.yaml

2020-02-09 14:43:30,786 Hello! This is Joey-NMT.
2020-02-09 14:43:49,425  dev bleu:  12.78 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-02-09 14:44:12,850 test bleu:  24.29 [Beam search decoding with beam size = 5 and alpha = 1.0]
